In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [2]:
clf = pd.read_csv("Data_for_practice/client_train.csv")

In [3]:
clf['creation_date'] = pd.to_datetime(clf['creation_date'])

In [4]:
clf["creation_date"].max()

Timestamp('2019-12-07 00:00:00')

In [5]:
clf['coop_time'] = (2021 - clf['creation_date'].dt.year)*12 - clf['creation_date'].dt.month

In [6]:
clf["coop_time"].max()

527

In [7]:
df = pd.read_csv("Data_for_practice/train_invoice.csv")

C:\Users\DELL\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
df['counter_type'] = df['counter_type'].map({"ELEC":1,"GAZ":0})

In [9]:
df['invoice_date'] = pd.to_datetime(df['invoice_date'], dayfirst=True)
df['invoice_month'] = df['invoice_date'].dt.month

In [10]:
df['delta_index'] = df['new_index'] - df['old_index']

In [13]:
df.tail()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,invoice_month,delta_index,delta_time
3279690,train_Client_67499,2018-06-25,11,6301121,0,410,6,1,402,0,0,0,143382,143784,4,1,6,402,120.0
3279691,train_Client_67499,2017-02-28,11,6301121,0,410,8,1,171,0,0,0,142871,143042,4,1,2,171,118.0
3279692,train_Client_67499,2016-01-11,11,6301121,0,410,8,1,51,0,0,0,142820,142871,2,1,1,51,127.0
3279693,train_Client_67499,2015-03-11,11,6301121,0,410,9,1,438,0,0,0,141762,142200,4,1,3,438,121.0
3279694,train_Client_67499,2017-06-23,11,6301121,0,410,8,1,124,0,0,0,143042,143166,4,1,6,124,7.0


In [12]:
df['delta_time'] = df.sort_values(['client_id','invoice_date']).groupby('client_id')['invoice_date'].diff(periods = 1).dt.days.reset_index(drop=True)

In [86]:
df["delta_time"] = df["delta_time"].fillna(0)

In [87]:
df["delta_time"].isna().value_counts()

False    3279695
Name: delta_time, dtype: int64

In [88]:
df = df.drop(columns = ["invoice_date","counter_number","counter_code","consommation_level_4","old_index","new_index"])

In [89]:
df.columns

Index(['client_id', 'tarif_type', 'counter_statue', 'reading_remarque',
       'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'months_number', 'counter_type',
       'invoice_month', 'delta_index', 'delta_time'],
      dtype='object')

In [90]:
df['counter_statue'] = df['counter_statue'].map({0:0,1:1,2:2,3:3,4:4,5:5,769:5,'0':0,'5':5,'1':1,'4':4,'A':0,618:5,269375:5,46:5,420:5})

In [91]:
 l  = ['tarif_type', 'counter_statue', 'reading_remarque','counter_coefficient', 'consommation_level_1', 'consommation_level_2','consommation_level_3', 'months_number', 'counter_type','invoice_month', 'delta_index', 'delta_time',]

In [95]:
df.groupby("client_id")["consommation_level_1"].std().isna().value_counts()

False    96312
True      3071
Name: consommation_level_1, dtype: int64

In [101]:
new_df = df.groupby('client_id')[l].agg(['mean','std'])
new_df.columns = ['_'.join(col).strip() for col in new_df.columns.values]
new_df.reset_index(inplace=True)

In [56]:
#new_df = df.groupby('client_id')[l].agg(["mean","std"])

In [100]:
df.groupby('client_id').size()

client_id
train_Client_0        35
train_Client_1        37
train_Client_10       18
train_Client_100      20
train_Client_1000     14
                      ..
train_Client_67495    55
train_Client_67496    39
train_Client_67497    18
train_Client_67498    21
train_Client_67499    14
Length: 99383, dtype: int64

In [104]:
dff = df.groupby('client_id').size().reset_index(name='transactions_count')

In [102]:
new_df.head()

,client_id,tarif_type_mean,tarif_type_std,counter_statue_mean,counter_statue_std,reading_remarque_mean,reading_remarque_std,counter_coefficient_mean,counter_coefficient_std,consommation_level_1_mean,...,months_number_mean,months_number_std,counter_type_mean,counter_type_std,invoice_month_mean,invoice_month_std,delta_index_mean,delta_index_std,delta_time_mean,delta_time_std
0,train_Client_0,11.0,0.0,0.0,0.0,6.971429,1.248192,1.0,0.0,352.400000,...,4.628571,2.101620,1.0,0.0,6.685714,3.587326,362.971429,341.553930,140.028571,89.676702
1,train_Client_1,11.0,0.0,0.0,0.0,7.216216,1.377097,1.0,0.0,557.540541,...,4.324324,1.375461,1.0,0.0,5.135135,3.128384,557.540541,197.935960,132.783784,93.917853
2,train_Client_10,11.0,0.0,0.0,0.0,7.055556,1.258955,1.0,0.0,798.611111,...,6.444444,3.399346,1.0,0.0,6.611111,3.292396,836.500000,646.808386,273.388889,252.605281
3,train_Client_100,11.0,0.0,0.0,0.0,6.150000,0.670820,1.0,0.0,1.200000,...,4.200000,0.615587,1.0,0.0,5.550000,3.203206,1.200000,3.607011,133.200000,98.162379
4,train_Client_1000,11.0,0.0,0.0,0.0,8.857143,0.363137,1.0,0.0,663.714286,...,3.714286,0.726273,1.0,0.0,7.571429,3.390355,922.642857,633.485669,113.214286,83.534588


In [268]:
full_df = pd.merge(clf, new_df, on='client_id', how='left')

In [269]:
full_df = pd.merge(full_df, dff, on='client_id', how='left')

In [270]:
full_df['invoice_per_cooperation'] = full_df['transactions_count'] / full_df['coop_time']

In [271]:
full_df = full_df.fillna(0)

In [272]:
full_df["invoice_month_std"].isna().value_counts()

False    99383
Name: invoice_month_std, dtype: int64

In [273]:
full_df = full_df.drop(columns = ["client_id","creation_date"])

In [274]:
for i in full_df.columns:
    factor = 5
    upper_lim = full_df[i].mean()+ full_df[i].std()*factor
    lower_lim = full_df[i].mean() - full_df[i].std()*factor
    full_df = full_df[(full_df[i]<upper_lim) & (full_df[i]>lower_lim)]

In [275]:
full_df.shape

(93121, 31)

In [276]:
full_df["target"].value_counts()

0    88062
1     5059
Name: target, dtype: int64

In [277]:
x = full_df

In [278]:
x = x.drop(columns = "target")

In [279]:
x.shape

(93121, 30)

In [280]:
x["delta_time_std"].isna().value_counts()

False    93121
Name: delta_time_std, dtype: int64

In [281]:
y = full_df["target"]

In [282]:
y.value_counts()

0    88062
1     5059
Name: target, dtype: int64

In [283]:
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size=1/5, random_state=1)

In [284]:
scaler = StandardScaler()
scaler.fit(x)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [285]:
pca = PCA(0.97)

In [286]:
pca.fit(x)

PCA(n_components=0.97)

In [287]:
pca.n_components_

7

In [288]:
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

In [253]:
model = RandomForestClassifier(n_estimators=100,criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',random_state=0, verbose=1)

In [254]:
model.fit(x_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.5s finished


RandomForestClassifier(random_state=0, verbose=1)

In [255]:
y_predict = model.predict(x_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


In [256]:
accuracy_score(y_predict,y_test)

0.9467919463087249

In [257]:
f1_score(y_test,y_predict,average="binary")

0.006018054162487463

In [258]:
roc_auc_score(y_test,y_predict)

0.5014822164863665

In [266]:
full_df.to_csv("first_reconizance_part1_train.csv")

In [267]:
full_df

,disrict,client_catg,region,target,coop_time,tarif_type_mean,tarif_type_std,counter_statue_mean,counter_statue_std,reading_remarque_mean,...,counter_type_mean,counter_type_std,invoice_month_mean,invoice_month_std,delta_index_mean,delta_index_std,delta_time_mean,delta_time_std,transactions_count,invoice_per_cooperation
0,60,11,101,0,312,11.000000,0.000000,0.000000,0.000000,6.971429,...,1.000000,0.000000,6.685714,3.587326,362.971429,341.553930,140.028571,89.676702,35,0.112179
1,69,11,107,0,223,11.000000,0.000000,0.000000,0.000000,7.216216,...,1.000000,0.000000,5.135135,3.128384,557.540541,197.935960,132.783784,93.917853,37,0.165919
2,62,11,301,0,417,11.000000,0.000000,0.000000,0.000000,7.055556,...,1.000000,0.000000,6.611111,3.292396,836.500000,646.808386,273.388889,252.605281,18,0.043165
3,69,11,105,0,289,11.000000,0.000000,0.000000,0.000000,6.150000,...,1.000000,0.000000,5.550000,3.203206,1.200000,3.607011,133.200000,98.162379,20,0.069204
4,62,11,303,0,74,11.000000,0.000000,0.000000,0.000000,8.857143,...,1.000000,0.000000,7.571429,3.390355,922.642857,633.485669,113.214286,83.534588,14,0.189189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99378,69,11,107,0,271,23.127273,14.436392,0.109091,0.685099,6.872727,...,0.581818,0.497807,6.890909,3.828126,783.418182,923.397424,90.272727,110.477491,55,0.202952
99379,63,11,312,1,278,11.000000,0.000000,0.000000,0.000000,6.282051,...,1.000000,0.000000,6.435897,3.234639,161.692308,82.672516,127.794872,61.458235,39,0.140288
99380,62,11,303,0,60,25.000000,15.434873,0.000000,0.000000,8.277778,...,0.500000,0.514496,7.777778,2.901431,544.444444,521.970838,64.444444,110.764840,18,0.300000
99381,63,11,311,0,192,17.904762,12.656638,0.000000,0.000000,6.142857,...,0.761905,0.436436,6.809524,2.803909,319.571429,319.975870,118.000000,139.089899,21,0.109375
